In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [2]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)

In [3]:
df_2020 = pd.read_csv('../../dobrano_2020_pre_preprocessado.csv', sep=';', low_memory=False, encoding='latin1')
df_2020.shape

(454432, 17)

In [4]:
df_2020.head()

,idade,sexo,raca_cor,escolaridade,local_ocorrencia,teve_assistencia_medica,dia_nasc,mes_nasc,ano_nasc,dia_obito,mes_obito,ano_obito,estado,grupo_causa_basica,causa_basica,ocupacao,grupo_ocupacao
0,68.0,M,BRANCA,EF_I,DOMICILIO,NAO,10,5.0,1951.0,25,1,2020,MG,Causas mal definidas e desconhecidas de mortal...,Outras causas mal definidas e as não especific...,NaN,TRABALHADORES EM SERVIÇOS DE REPARAÇÃO E MANUT...
1,53.0,M,PRETA,EF_I,DOMICILIO,NAO,16,3.0,1966.0,25,1,2020,MG,Causas mal definidas e desconhecidas de mortal...,Outras causas mal definidas e as não especific...,Trabalhador agropecuário em geral,"TRABALHADORES AGROPECUÁRIOS, FLORESTAIS E DA P..."
2,82.0,F,BRANCA,EF_I,HOSPITAL,NaN,10,7.0,1937.0,14,1,2020,MG,Outras doenças dos intestinos,"Doença diverticular do intestino, de localizaç...",NaN,TRABALHADORES EM SERVIÇOS DE REPARAÇÃO E MANUT...
3,73.0,F,BRANCA,EF_I,HOSPITAL,SIM,2,11.0,1946.0,20,1,2020,MG,Outras doenças do aparelho respiratório,Insuficiência respiratória aguda,NaN,TRABALHADORES EM SERVIÇOS DE REPARAÇÃO E MANUT...
4,60.0,M,BRANCA,SUPERIOR_C,HOSPITAL,SIM,12,9.0,1959.0,26,1,2020,MG,Outras doenças bacterianas,Septicemia não especificada,Professor da educação de jovens e adultos do ...,PROFISSIONAIS DAS CIÊNCIAS E DAS ARTES ...


#### TRANSFORMANDO DADOS EM NUMÉRICOS

In [6]:
df_2020['raca_cor'] = df_2020['raca_cor'].replace(['BRANCA', 'PRETA', 'AMARELA', 'PARDA', 'INDIGENA'], [1, 2, 3, 4, 5])

In [7]:
df_2020['escolaridade'] = df_2020['escolaridade'].replace(
                                          ['SEM', 'EF_I', 'EF_II', 'EM', 'SUPERIOR_I', 'SUPERIOR_C', 'I'],
                                        [0, 1, 2, 3, 4, 5, 9]
                                         )

In [8]:
df_2020['local_ocorrencia'] = df_2020['local_ocorrencia'].replace(['HOSPITAL', 'OES', 'DOMICILIO', 'VIAP', 'OUTROS', 'I'],
                                            [1, 2, 3, 4, 5, 9]
                                         )

In [9]:
df_2020['teve_assistencia_medica'] = df_2020['teve_assistencia_medica'].replace(['SIM', 'NAO', 'I'], [1, 2, 9]
                                             )

In [10]:
df_2020['sexo'] = df_2020['sexo'].replace(['M', 'F', 'I'], [1, 2, 0]) 

In [11]:
df_cid_grupos = pd.read_csv('../../CID-10-GRUPOS.csv', header=0, sep=';', encoding='latin1')
# df_cid_grupos.head()

df_cid_grupos = df_cid_grupos.drop(columns=['Unnamed: 0'])

In [15]:
df_cid_grupos.head()

,CATINIC,CATFIM,DESCRICAO,DESCRABREV,VALOR
0,A00,A09,Doenças infecciosas intestinais,Doenças infecciosas intestinais,0
1,A15,A19,Tuberculose,Tuberculose,1
2,A20,A28,Algumas doenças bacterianas zoonóticas,Algumas doenças bacterianas zoonóticas,2
3,A30,A49,Outras doenças bacterianas,Outras doenças bacterianas,3
4,A50,A64,Infecções de transmissão predominantemente sexual,Infecções de transmissão predominantemente sexual,4


In [13]:
cid_grupos = [i for i in range(df_cid_grupos.shape[0])]

In [14]:
df_cid_grupos['VALOR'] = cid_grupos

In [16]:
df_ocup_grupo = pd.read_csv('../../Downloads/CBO2002_Grande_Grupo.csv', header=0, sep=';', encoding='latin1')
ocup_grupos = [i for i in range(df_cid_grupos.shape[0])]

In [ ]:
def categoria_cid(row, cids):
    cids.loc[((cids.CAUSABAS.str.len() == 3) & (cids.CAUSABAS == row.CAT)),
             ['CAUSA_BASICA']] = row.DESCRICAO 

In [ ]:
df_2020['grupo_causa_basica'] = df_2020['grupo_causa_basica'].replace(df_cid_grupos['DESCRICAO'].values, df_cid)

#### SEPRANDO DATASETS

In [10]:
df_2020 = df_2020.loc[df_2020['CAUSA_BASICA'].notnull()]

In [12]:
df_2020 = df_2020.drop(columns=['FONTE', 'CIRCOBITO'])

In [14]:
df_2020.loc[((df_2020['OCUPACAO'].isnull()) & (df_2020['GRUPO_OCUPACAO'].isnull())), ['GRUPO_OCUPACAO']] = 'SEM OCUPAÇÃO'
    
df_2020.loc[((df_2020['OCUPACAO'].isnull()) & (df_2020['GRUPO_OCUPACAO'] == 'SEM OCUPAÇÃO')), ['OCUPACAO']] = 'SEM OCUPAÇÃO'
    
df_2020.loc[((df_2020['OCUPACAO'].isnull()) & (df_2020['GRUPO_OCUPACAO'] != 'SEM OCUPAÇÃO')), ['OCUPACAO']] = 'NÃO ESPECIFICADO'
    

In [16]:
y = df_2020['CAUSA_BASICA']

In [18]:
x = df_2020.drop(columns=['CAUSA_BASICA'])

In [21]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state = 42)

In [22]:
x_train.shape

(374979, 18)

In [23]:
y_test.shape

(124993,)

In [25]:
rf = RandomForestClassifier(n_estimators = 1000, random_state = 42)

In [27]:

rf.fit(x_train, y_train)

ValueError: could not convert string to float: 'F'